In [1]:
# Loading packages

# Path and file, data handling
import os
import h5py
import numpy as np
import pandas as pd
from pathlib import Path

# Image preprocessing
from skimage.io import imread
import cv2

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Splitting dataset
from sklearn.model_selection import train_test_split

# Image Augmentation
import imgaug as aug
import imgaug.augmenters as iaa

# Deep Learning
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [2]:
data_dir = Path('D:/Projects/Covid19/data/')

train_dir = data_dir / 'Original_cropped'

train_dir

WindowsPath('D:/Projects/Covid19/data/Original_cropped')

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=0.2) # set validation split
    
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=4,
    color_mode="rgb",
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(256, 256),
    batch_size=4,
    color_mode="rgb",
    class_mode='categorical',
    subset='validation') # set as validation data

Found 1292 images belonging to 3 classes.
Found 322 images belonging to 3 classes.


In [4]:
epochs=50
based_model_last_block_layer_number = 418
train_steps = train_generator.samples//train_generator.batch_size
val_steps = validation_generator.samples//validation_generator.batch_size

In [5]:
# Densenet121

base_model = DenseNet121(input_shape = (256, 256, 3), weights='imagenet', include_top=False)

multi_class_disease = base_model.output
multi_class_disease = GlobalAveragePooling2D()(multi_class_disease)
multi_class_disease = Dense(3, activation='softmax')(multi_class_disease)

multi_class_disease = Model(base_model.input, multi_class_disease)
multi_class_disease.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [6]:
multi_class_disease.compile(optimizer='nadam',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])


In [7]:
for layer in multi_class_disease.layers:
    layer.trainable = False

In [8]:
top_weights_path = os.path.join(os.path.abspath("D:/Projects/Covid19/Densenet121/"), 'top_model_weights.h5')

callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_accuracy', patience=5, verbose=0)
]

In [9]:
# training model

history = multi_class_disease.fit_generator(train_generator,
                                            steps_per_epoch=train_steps,
                                            epochs=epochs,
                                            validation_data=validation_generator,
                                            validation_steps=val_steps,
                                            callbacks=callbacks_list)

Epoch 1/50
166/323 [==============>...............] - ETA: 2:28 - loss: 1.1537 - accuracy: 0.4789

KeyboardInterrupt: 

In [ ]:
print("\nStarting to Fine Tune Model\n")
# set weights to imagenet 

In [ ]:
multi_class_disease.load_weights(top_weights_path)

In [ ]:
# Compiling with Adam optimizer

multi_class_disease.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
for layer in multi_class_disease.layers[:based_model_last_block_layer_number]:
    layer.trainable = False
for layer in multi_class_disease.layers[based_model_last_block_layer_number:]:
    layer.trainable = True

In [ ]:
# save weights of best training epoch: monitor either val_loss or val_acc

final_weights_path = os.path.join(os.path.abspath("D:/Projects/Covid19/Densenet121/"), 'densenet121_model_weights.h5')

callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_accuracy', patience=5, verbose=0)
]

In [ ]:
history2 = multi_class_disease.fit_generator(train_generator,
                                            steps_per_epoch=train_steps,
                                            epochs=epochs,
                                            validation_data=validation_generator,
                                            validation_steps=val_steps,
                                            callbacks=callbacks_list)

In [ ]:

model_json = multi_class_disease.to_json()
with open(os.path.join('D:/Projects/Covid19/Densenet121/', 'densenet121_model.json'), 'w') as json_file:
    json_file.write(model_json)